In [2]:
import os
import pandas as pd

input_directory = "runs/"
cats = [c for c in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, c))]
finalpop_dataframes = []

s_queue = 0
a_queue = 0
s_incomplete = 0
a_incomplete = 0
r_queue = 0
r_incomplete = 0

for cat in cats:
    cat_dir = os.path.join(input_directory, cat)
    for rundir in os.listdir(cat_dir):
        with open(os.path.join(cat_dir, rundir, "metadata.csv")) as fh:
            lines = [line[:-1].split(';') for line in fh.readlines()[1:-3]]
            settings = dict(lines)
            settings['age_regularization'] = float(settings['age_regularization'])
            settings['subset'] = float(settings['subset'])
            if settings['subset'] == 0.7:
                s_queue += 1
            elif settings['age_regularization'] == 50:
                a_queue += 1
            else:
                r_queue += 1
        if not os.path.exists(os.path.join(cat_dir, rundir, "final_pareto.csv")):
            if settings['subset'] == 0.7:
                s_incomplete += 1
            elif settings['age_regularization'] == 50:
                a_incomplete += 1
            else:
                r_incomplete += 1
            continue
        df = pd.read_csv(
            filepath_or_buffer=os.path.join(cat_dir, rundir, "final_pareto.csv"), 
            sep=';',
            header=0,
            #names=["run", "task", "sample", "score", "length", "expression"]
        )
        with open(os.path.join(cat_dir, rundir, "metadata.csv")) as fh:
            lines = [line[:-1].split(';') for line in fh.readlines()[1:-3]]
            settings = dict(lines)
        df['age'] = settings['age_regularization']
        df['subset'] = settings['subset']
        df['problem'] = settings['problem'][4:]
        finalpop_dataframes.append(df)

final_populations = pd.concat(finalpop_dataframes)
final_populations['length'] = final_populations['length'].astype(float)
final_populations['age'] = final_populations['age'].astype(float)
final_populations['subset'] = final_populations['subset'].astype(float)

final_populations['type'] = 'regular'
final_populations['type'].loc[final_populations['age'] == 50] = 'age'
final_populations['type'].loc[final_populations['subset'] == 0.7] = 'subset'

e:\repositories\symbolicdefaults\venv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
e:\repositories\symbolicdefaults\venv\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [3]:
final_populations['type'].value_counts()

regular    240429
subset      32350
Name: type, dtype: int64

In [4]:
s_queue, s_incomplete, a_queue, a_incomplete , r_queue, r_incomplete

(390, 0, 0, 0, 1000, 0)

In [6]:
final_populations = final_populations[final_populations['type'] == 'regular']

In [7]:
final_populations

,run,k_tasks,task,sample,score,length,expression,age,subset,problem,type
0,00129af0-4ff2-4845-8850-3f432eaa56ae,7592,0,in,0.9359,7.0,"make_tuple(if_gt(2, truediv(13, po), 7, 83), m...",100000.0,1.0,svm,regular
1,00129af0-4ff2-4845-8850-3f432eaa56ae,7592,9978,val,0.9584,7.0,"make_tuple(if_gt(2, truediv(13, po), 7, 83), m...",100000.0,1.0,svm,regular
2,00129af0-4ff2-4845-8850-3f432eaa56ae,7592,3560,val,0.4071,7.0,"make_tuple(if_gt(2, truediv(13, po), 7, 83), m...",100000.0,1.0,svm,regular
3,00129af0-4ff2-4845-8850-3f432eaa56ae,7592,145681,val,0.6213,7.0,"make_tuple(if_gt(2, truediv(13, po), 7, 83), m...",100000.0,1.0,svm,regular
4,00129af0-4ff2-4845-8850-3f432eaa56ae,7592,15,val,0.9726,7.0,"make_tuple(if_gt(2, truediv(13, po), 7, 83), m...",100000.0,1.0,svm,regular
...,...,...,...,...,...,...,...,...,...,...,...
247,ffbc80f1-dde5-41bf-8844-744022e91596,167140,9950,val,0.8387,1.0,"make_tuple(p, mkd)",100000.0,1.0,svm,regular
248,ffbc80f1-dde5-41bf-8844-744022e91596,167140,189928,val,0.9284,1.0,"make_tuple(p, mkd)",100000.0,1.0,svm,regular
249,ffbc80f1-dde5-41bf-8844-744022e91596,167140,2074,val,0.9820,1.0,"make_tuple(p, mkd)",100000.0,1.0,svm,regular
250,ffbc80f1-dde5-41bf-8844-744022e91596,167140,3493,val,0.1400,1.0,"make_tuple(p, mkd)",100000.0,1.0,svm,regular


# Selecting from the Pareto Front

In [40]:
selected_solutions = pd.DataFrame([], list(final_populations.columns))

In [113]:
from collections import defaultdict

delta = 0.01
solution_per_task = dict()
validation_results = final_populations[final_populations['sample'] == 'val']
test_results = final_populations[final_populations['sample'] == 'test']
test_scores = []

def select_best(df, max_len=None, delta=0):
    best_so_far = 0
    expression = None
    for i, row in run_results.sort_values(by='length').iterrows():
        if row['score'] > best_so_far + delta and (max_len is None or row['length'] <= max_len):
            best_so_far = row['score']
            expression = row['expression']
    return expression

for run in validation_results.run.unique():
    run_results = validation_results[validation_results.run == run]
    best_so_far = 0
    expression = None
    
    # task, expression and score
    for name, args in [
        ("best", [ None, 0]),
        ("three-d0005", [ 3, 0.005]),
        ("d001", [ 3, 0.005]),
        ("len1", [ 1, 0]),
    ]:        
        expression = select_best(run_results, *args)
        result = test_results[(test_results.run == run) & (test_results.expression == expression)]
        test_scores.append(dict(
            task=result.task.iloc[0],
            score=result.score.iloc[0],
            expression=name
        ))


In [111]:
# note,  we have some missing results (1000 expected)
sum(len(v) for t, v in solutions_by_task.items())

977

# Baseline Performance

In [114]:
import os
import pandas as pd

input_directory = "runs/"
cats = [c for c in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, c))]
benchmark = pd.DataFrame([], columns=list(final_populations.columns))


for cat in cats:
    cat_dir = os.path.join(input_directory, cat)
    for rundir in os.listdir(cat_dir):
        with open(os.path.join(cat_dir, rundir, "evaluations.csv")) as fh:
            lines = fh.readlines()
        no_benchmarks = 1 if cat.startswith('rf') else 3
        for line in lines[-no_benchmarks:]:
            run, task_str, task, _, sample, score, length, _, name = line[:-1].split(';')
            if sample == 'test':
                benchmark = benchmark.append(dict(
                    run=run,
                    k_tasks=task_str,
                    task=int(task),
                    sample=sample,
                    score=float(score),
                    length=int(length),
                    expression=name,
                    age=0,
                    subset=0,
                    problem=cat,
                    type='regular',
                ), ignore_index=True)



In [126]:
found_scores = pd.DataFrame(test_scores).groupby(by=["task", "expression"], as_index=False).mean()

# Comparison

In [115]:
solutions = benchmark[benchmark.problem.str.startswith('svm')].groupby(["task", "expression"]).mean().reset_index()


In [95]:
solutions = benchmark[benchmark.problem.str.startswith('svm')].groupby(["task", "expression"]).mean().reset_index()
for found_expression in test_scores:
    solutions = solutions.append(dict(
        task=task,
        expression="found",
        score=sum(scores)/len(scores),
    ), ignore_index=True)
solutions

,task,expression,score
0,3,mlr_default,0.8768
1,3,sklearn_default,0.9168
2,3,symbolic_best,0.9764
3,6,mlr_default,0.9805
4,6,sklearn_default,0.9805
...,...,...,...
395,167140,found,0.9585
396,167141,found,0.9960
397,189924,found,0.6859
398,37,found,0.8817


In [129]:
all_solutions = pd.concat([found_scores, solutions])

## Average Score and Rank Across All Tasks

In [130]:
found_scores.expression.unique()

array(['best', 'd001', 'len1', 'three-d0005'], dtype=object)

In [131]:
from IPython.display import display
for strategy in found_scores.expression.unique():
    strategy_solutions = all_solutions[all_solutions.expression.isin([strategy, 'mlr_default', 'sklearn_default', 'symbolic_best'])]
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html
    strategy_solutions["rank"] = strategy_solutions.groupby("task")["score"].rank("average", ascending=False)
    display(strategy_solutions.groupby("expression").mean())

e:\repositories\symbolicdefaults\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,task,score,rank
expression,,,
best,52959.72,0.893609,2.33
mlr_default,52959.72,0.879356,2.96
sklearn_default,52959.72,0.881658,2.79
symbolic_best,52959.72,0.899472,1.92


e:\repositories\symbolicdefaults\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,task,score,rank
expression,,,
d001,52959.72,0.885756,2.43
mlr_default,52959.72,0.879356,2.89
sklearn_default,52959.72,0.881658,2.73
symbolic_best,52959.72,0.899472,1.95


e:\repositories\symbolicdefaults\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,task,score,rank
expression,,,
len1,52959.72,0.888698,2.41
mlr_default,52959.72,0.879356,2.90
sklearn_default,52959.72,0.881658,2.75
symbolic_best,52959.72,0.899472,1.94


e:\repositories\symbolicdefaults\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,task,score,rank
expression,,,
mlr_default,52959.72,0.879356,2.89
sklearn_default,52959.72,0.881658,2.73
symbolic_best,52959.72,0.899472,1.95
three-d0005,52959.72,0.885756,2.43


In [106]:
solutions.groupby("expression").mean()

,task,score,rank
expression,,,
found,52959.72,0.888698,2.41
mlr_default,52959.72,0.879356,2.90
sklearn_default,52959.72,0.881658,2.75
symbolic_best,52959.72,0.899472,1.94


# Which method regularizes size best?

In [8]:
final_populations.groupby(["problem", "k_tasks", "sample"]).median()

score  length       age  subset
problem k_tasks sample                                  
svm     3       in      0.9326     7.0  100000.0     1.0
                test    0.9505     7.0  100000.0     1.0
                val     0.9572     7.0  100000.0     1.0
        6       in      0.9292     7.0  100000.0     1.0
                test    0.9849     7.0  100000.0     1.0
...                        ...     ...       ...     ...
        189928  test    0.9727     6.0  100000.0     1.0
                val     0.8932     6.0  100000.0     1.0
        190411  in      0.9338     6.0  100000.0     1.0
                test    0.9087     6.0  100000.0     1.0
                val     0.9432     6.0  100000.0     1.0

[300 rows x 4 columns]

In [22]:
import os
import pandas as pd

input_directory = "runs/"
runs = os.listdir(input_directory)
finalpop_dataframes = []

for rundir in runs:
    df = pd.read_csv(
        filepath_or_buffer=os.path.join(input_directory, rundir, "finalpops.csv"), 
        sep=';',
        header=0,
    )
    finalpop_dataframes.append(df)

final_populations = pd.concat(finalpop_dataframes)

In [23]:
final_populations

,run,k_tasks,task,sample,score,length,expression
0,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",0,in,0.8892,1,"make_tuple(p, mkd)"
1,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",28,out,0.9991,1,"make_tuple(p, mkd)"
2,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",53,out,0.9586,1,"make_tuple(p, mkd)"
3,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",168911,out,0.9462,1,"make_tuple(p, mkd)"
4,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",0,in,0.9007,2,"make_tuple(m, truediv(mkd, xvar))"
...,...,...,...,...,...,...,...
75,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",168911,out,0.9526,5,"make_tuple(truediv(m, xvar), min(truediv(mkd, ..."
76,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",0,in,0.8847,2,"make_tuple(add(xvar, po), mkd)"
77,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",28,out,0.9991,2,"make_tuple(add(xvar, po), mkd)"
78,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",53,out,0.9580,2,"make_tuple(add(xvar, po), mkd)"


In [24]:
import os
import pandas as pd

input_directory = "runs/"
runs = os.listdir(input_directory)
finalpop_dataframes = []

for rundir in runs:
    df = pd.read_csv(
        filepath_or_buffer=os.path.join(input_directory, rundir, "evaluations.csv"), 
        sep=';',
        header=0,
    )
    finalpop_dataframes.append(df)

final_populations = pd.concat(finalpop_dataframes)

In [25]:
final_populations

,run,k_tasks,task,gen,inout,score,length,endresult,expression
0,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",0,9,in,0.8892,1,True,"make_tuple(p, mkd)"
1,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",0,9,in,0.9007,2,True,"make_tuple(m, truediv(mkd, xvar))"
2,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",0,9,in,0.9007,3,True,"make_tuple(m, min(truediv(mkd, xvar), mcp))"
3,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",28,9,out,0.9991,1,True,"make_tuple(p, mkd)"
4,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",28,9,out,0.9995,2,True,"make_tuple(m, truediv(mkd, xvar))"
5,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",28,9,out,0.9995,3,True,"make_tuple(m, min(truediv(mkd, xvar), mcp))"
6,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",53,9,out,0.9586,1,True,"make_tuple(p, mkd)"
7,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",53,9,out,0.9020,2,True,"make_tuple(m, truediv(mkd, xvar))"
8,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",53,9,out,0.9020,3,True,"make_tuple(m, min(truediv(mkd, xvar), mcp))"
9,4ef06b81-a017-476a-bfcd-42b5f4071e3a,"28,53,168911",168911,9,out,0.9462,1,True,"make_tuple(p, mkd)"
